In [ ]:
######## NRoot3 by NRoot3 supercell #########

import numpy as np
from pymatgen.core.structure import Structure
from pymatgen.io.vasp.inputs import Poscar
import os

def create_all_nroot3_supercells(poscar_file='POSCAR1.vasp', n=1):
    """
    Create all four types of n√3×n√3 supercells for a given integer n,
    only transforming the x-y plane (keeping z-direction unchanged).
    """
    # Load the original structure
    structure = Structure.from_file(poscar_file)
    
    # Verify the structure is hexagonal
    a, b, c = structure.lattice.abc
    alpha, beta, gamma = structure.lattice.angles
    if not (abs(a - b) < 0.1 and abs(gamma - 120) < 1):
        print("Warning: The input structure doesn't appear to be hexagonal (a ≈ b, γ ≈ 120°)")
    
    # Define all possible √3×√3 transformation matrices (2D only)
    base_matrices_2d = {
        1: np.array([[1, 1], [-2, 1]]),
        2: np.array([[1, -2], [1, 1]]),
        3: np.array([[2, -1], [1, 1]]),
        4: np.array([[1, 2], [-1, 1]])
    }
    
    # Create output directory
    output_dir = f'n{n}root3_supercells'
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"\nGenerating n√3×n√3 supercells for n={n} (x-y plane only)...")
    print(f"Original structure: {len(structure)} atoms")
    print(f"Lattice parameters (Å): a={a:.3f}, b={b:.3f}, c={c:.3f}")
    print(f"Lattice angles (°): α={alpha:.1f}, β={beta:.1f}, γ={gamma:.1f}\n")
    
    for matrix_type, base_2d in base_matrices_2d.items():
        # Create 3D transformation matrix (identity for z-direction)
        transformation = np.eye(3)
        transformation[:2,:2] = n * base_2d  # Only modify x-y components
        
        # Create the supercell
        supercell = structure.copy()
        supercell.make_supercell(transformation)
        
        # Calculate scaling factor (should be n√3×n√3 in plane)
        det_2d = np.linalg.det(n * base_2d)
        scaling_factor = np.sqrt(det_2d)
        
        # Write the new POSCAR
        output_file = os.path.join(output_dir, f'POSCAR_n{n}root3_type{matrix_type}')
        poscar = Poscar(supercell)
        poscar.write_file(output_file)
        
        # Print information
        new_a, new_b, new_c = supercell.lattice.abc
        print(f"Type {matrix_type}:")
        print(f"  2D Transformation matrix:\n{base_2d}")
        print(f"  Full 3D transformation:\n{transformation}")
        print(f"  Area scaling: {det_2d:.1f}x (theoretical: {3*n**2:.1f}x)")
        print(f"  New atoms: {len(supercell)}")
        print(f"  New lattice (Å): a={new_a:.3f}, b={new_b:.3f}, c={new_c:.3f}")
        print(f"  New angles (°): γ={supercell.lattice.angles[2]:.1f}")
        print(f"  Output: {output_file}\n")

if __name__ == "__main__":
    # Get user input for n value
    while True:
        try:
            n = 5
            if n > 0:
                break
            print("Please enter a positive integer.")
        except ValueError:
            print("Please enter a valid integer.")
    
    create_all_nroot3_supercells(n=n)


Generating n√3×n√3 supercells for n=5 (x-y plane only)...
Original structure: 2 atoms
Lattice parameters (Å): a=2.467, b=2.467, c=7.803
Lattice angles (°): α=90.0, β=90.0, γ=60.0

Type 1:
  2D Transformation matrix:
[[ 1  1]
 [-2  1]]
  Full 3D transformation:
[[  5.   5.   0.]
 [-10.   5.   0.]
 [  0.   0.   1.]]
  Area scaling: 75.0x (theoretical: 75.0x)
  New atoms: 150
  New lattice (Å): a=21.367, b=21.367, c=7.803
  New angles (°): γ=120.0
  Output: n5root3_supercells\POSCAR_n5root3_type1

Type 2:
  2D Transformation matrix:
[[ 1 -2]
 [ 1  1]]
  Full 3D transformation:
[[  5. -10.   0.]
 [  5.   5.   0.]
 [  0.   0.   1.]]
  Area scaling: 75.0x (theoretical: 75.0x)
  New atoms: 150
  New lattice (Å): a=21.367, b=21.367, c=7.803
  New angles (°): γ=120.0
  Output: n5root3_supercells\POSCAR_n5root3_type2

Type 3:
  2D Transformation matrix:
[[ 2 -1]
 [ 1  1]]
  Full 3D transformation:
[[10. -5.  0.]
 [ 5.  5.  0.]
 [ 0.  0.  1.]]
  Area scaling: 75.0x (theoretical: 75.0x)
  New ato

In [ ]:
######### Root3 by Root3 #########

import numpy as np
from pymatgen.core.structure import Structure
from pymatgen.io.vasp.inputs import Poscar

def create_root3_supercell(poscar_file='POSCAR1.vasp', output_file='POSCAR_root3', matrix_type=1):
    """
    Create a √3×√3 supercell from a hexagonal POSCAR file with different matrix options.
    
    Args:
        poscar_file (str): Input POSCAR filename
        output_file (str): Output POSCAR filename
        matrix_type (int): Which transformation matrix to use (1-4)
    """
    # Load the original structure
    structure = Structure.from_file(poscar_file)
    
    # Verify the structure is hexagonal
    a, b, c = structure.lattice.abc
    alpha, beta, gamma = structure.lattice.angles
    if not (abs(a - b) < 0.1 and abs(gamma - 120) < 1):
        print("Warning: The input structure doesn't appear to be hexagonal (a ≈ b, γ ≈ 120°)")
    
    # Define all possible √3×√3 transformation matrices with determinant exactly 3
    matrices = {
        1: np.array([[1, 1, 0], [-2, 1, 0], [0, 0, 1]]),  # det = (1*1) - (1*-2) = 1 + 2 = 3
        2: np.array([[1, -2, 0], [1, 1, 0], [0, 0, 1]]),   # det = (1*1) - (-2*1) = 1 + 2 = 3
        3: np.array([[2, -1, 0], [1, 1, 0], [0, 0, 1]]),   # det = (2*1) - (-1*1) = 2 + 1 = 3
        4: np.array([[1, 2, 0], [-1, 1, 0], [0, 0, 1]])    # det = (1*1) - (2*-1) = 1 + 2 = 3
    }
    
    # Get the selected transformation matrix
    if matrix_type not in matrices:
        raise ValueError("matrix_type must be between 1 and 4")
    transformation = matrices[matrix_type]
    
    # Create the supercell
    supercell = structure.copy()
    supercell.make_supercell(transformation)
    
    # Write the new POSCAR
    poscar = Poscar(supercell)
    poscar.write_file(output_file)
    
    print(f"√3×√3 supercell created with {len(supercell)} atoms using matrix type {matrix_type}.")
    print(f"Transformation matrix:\n{transformation}")
    print(f"New POSCAR written to: {output_file}")

if __name__ == "__main__":
    # Example usage with different matrix types
    for i in range(1, 5):
        create_root3_supercell(output_file=f'POSCAR_root3_type{i}', matrix_type=i)

√3×√3 supercell created with 6 atoms using matrix type 1.
Transformation matrix:
[[ 1  1  0]
 [-2  1  0]
 [ 0  0  1]]
New POSCAR written to: POSCAR_root3_type1
√3×√3 supercell created with 6 atoms using matrix type 2.
Transformation matrix:
[[ 1 -2  0]
 [ 1  1  0]
 [ 0  0  1]]
New POSCAR written to: POSCAR_root3_type2
√3×√3 supercell created with 6 atoms using matrix type 3.
Transformation matrix:
[[ 2 -1  0]
 [ 1  1  0]
 [ 0  0  1]]
New POSCAR written to: POSCAR_root3_type3
√3×√3 supercell created with 6 atoms using matrix type 4.
Transformation matrix:
[[ 1  2  0]
 [-1  1  0]
 [ 0  0  1]]
New POSCAR written to: POSCAR_root3_type4
